In [ ]:
# default_exp utils

# utils
> Provides different util functions

In [ ]:
#export
import json
from copy import deepcopy
import numpy as np
from PIL import Image

from icevision.core.mask import EncodedRLEs, MaskArray

from pycocotools import mask as mask_utils

## Test data setup

In [ ]:
import icedata
from icevision.data.data_splitter import SingleSplitSplitter

In [ ]:
test_data_path_instance_segmentation = icedata.pennfudan.load_data()
test_instance_segmentation_parser = icedata.pennfudan.parser(data_dir=test_data_path_instance_segmentation)
test_instance_segmentation_records = test_instance_segmentation_parser.parse(SingleSplitSplitter())[0]
test_instance_segmentation_class_map = test_instance_segmentation_records[0].detection.class_map

## Instance segmentation

In [ ]:
#export
def erles_to_string(erles):
    erles_copy = deepcopy(erles)
    erles_copy["counts"] = erles_copy["counts"].decode("utf-8")
    return json.dumps(erles_copy)

In [ ]:
#hide
test_erles = test_instance_segmentation_records[0].as_dict()["detection"]["masks"][0].to_erles(None, None).erles
test_string_erles = erles_to_string(test_erles[0])
assert test_string_erles == '{"size": [536, 559], "counts": "ecc22g`00O2O0O1O100O1O00100O001O10O01O1O0010OO2N2M2O2M2O2M2O2M2O2N1N3N1N3N1N3N0O01O01O00000O1000000O2O001`NbNfC^1Z<dNcC^1\\\\<eNaC[1_<gN_CZ1`<iN\\\\CX1e<iNXCY1h<iNUCX1o9cNhG6VNX1l9mNjGLXNX1n9oNhGHXNZ1Q:ROcGDZN\\\\1R:SObGBZN[1U:UO_G@ZN]1W:TO_G^OYN^1Y:UO]G]OXN`1[:UO[G[OXNa1^:UOYG[OWNa1`:VOXGXOVNc1c:VOVGWOUNd1g:UOSGWOTNf1j:SOQGWOTNf1m:SOoF[1S9dNhF`1Z9`NVFo1k9QNhEZ2Z:iMVEb2l:d11N2N2O1N3M3N2M3M3N2M2O200YKbDS4R<01O1O10O4L3N3L3M4M2M4ZE\\\\Ko8d4PG^Ko8b4PG^KR9`4nF`KS9_4lFbKU9]4kFcKX9Z4hFeK]9W4cFiKb9V4ZFjKj9V4QFkKT:T4gEmK]:h1jD6d0SNh:a1hD<:UNX;T1bDh01UNf;i0^DR1GVNU<>WD]1_OVNc<3SDU2W<`MmC]2Y=N3M2N3M2N3M2N3M2N3M3M3M3M3M3M2N3L4M3M3M3M6J5K6J6J^SV4"}'

In [ ]:
#export
def erles_to_counts_to_utf8(erles):
    erles_copy = deepcopy(erles)
    for entry in erles_copy:
        entry["counts"] = entry["counts"].decode("utf-8")
    return erles_copy

In [ ]:
#hide
test_erles_with_utf_8_counts = erles_to_counts_to_utf8(test_erles)
for erles in test_erles_with_utf_8_counts:
    assert isinstance(erles["counts"], str)

In [ ]:
#export
def string_to_erles(erles_string):
    erles = json.loads(erles_string)
    erles["counts"] = erles["counts"].encode()
    return erles

In [ ]:
#hide
erles_string = json.dumps(erles_to_counts_to_utf8(test_erles)[0])
test_erles_from_string = string_to_erles(erles_string)
assert isinstance(test_erles_from_string["counts"], bytes)

In [ ]:
#export
def correct_mask(mask_array, pad_x, pad_y, width, height):
    # correct mask
    corrected_mask_array = mask_array.transpose(2, 0, 1)
    if round(pad_x/2) > 0:
        corrected_mask_array=corrected_mask_array[:,:,round(pad_x/2):round(-pad_x/2)]
    if round(pad_y/2) > 0:
        corrected_mask_array=corrected_mask_array[:,round(pad_y/2):round(-pad_y/2),:]
    corrected_mask_array = np.array(Image.fromarray(corrected_mask_array[0,:,:]).resize([width, height], Image.NEAREST))
    corrected_mask_array = np.expand_dims(corrected_mask_array, 0)
    # convert mask array to mask and get erles (only one erles exist!)
    corrected_mask = MaskArray(corrected_mask_array)
    return corrected_mask

In [ ]:
#hide
test_mask = np.zeros([1, 30, 30])
test_mask_corrected = correct_mask(test_mask, 10, 0, 30, 20)
assert test_mask_corrected.data.shape == (1, 20, 30)

In [ ]:
#export
def decorrect_mask(mask_array, pad_x, pad_y, width, height):
    corrected_mask_array = mask_array.transpose(2, 0, 1)
    # resize
    corrected_mask_array = np.array(Image.fromarray(corrected_mask_array[0,:,:]).resize([width, height], Image.NEAREST))
    corrected_mask_array = np.expand_dims(corrected_mask_array, 0)
    # pad
    corrected_mask_array = np.pad(corrected_mask_array, [[0,0], [pad_y, pad_y], [pad_x, pad_x],])
    corrected_mask = MaskArray(corrected_mask_array)
    return corrected_mask

In [ ]:
test_mask = np.ones([1, 10,10])
test_mask_decorrected = decorrect_mask(test_mask, 1, 2, 5, 5)
assert test_mask_decorrected.shape == (1,9,7)